In [1]:
# Célula 1: Importações
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceEmbeddings
# >>> ADICIONE ESTA LINHA ABAIXO <<<
from langchain_community.llms import LlamaCpp
# Para usar SystemMessage e HumanMessage (embora não estamos usando diretamente com LlamaCpp.invoke)
from langchain.schema import HumanMessage, SystemMessage
from IPython.display import Markdown # Se você estiver usando Jupyter/IPython
import os

In [2]:
# Célula 2: Carregamento do Documento
path_txt = "/home/laza/chatbot_rag_hf/base_conhecimento.txt"
loader = TextLoader(path_txt, encoding="utf-8")
docs = loader.load()

In [3]:
# Célula 3: Embeddings e Qdrant
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

qdrant = Qdrant.from_documents(
    documents=docs,
    embedding=embeddings,
    location=":memory:",
    collection_name="criterios_mat"
)

/tmp/ipykernel_111684/1038710504.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
/home/laza/chatbot_rag_hf/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Célula 4: Função custom_prompt
def custom_prompt(query: str):
    results = qdrant.similarity_search(query, k=5)
    source_knowledge = "\n\n".join([doc.page_content for doc in results])
    prompt = f"""
Você é um avaliador pedagógico da área de Matemática. Responda com base apenas no conteúdo fornecido.

Contexto:
{source_knowledge}

Pergunta:
{query}
""".strip()
    return prompt

In [ ]:
# Célula 5: Configuração e Função Chatbot (Adaptada para LlamaCpp/GGUF Local)

import os
import shutil # Para obter a largura do terminal

print("Carregando modelo GGUF localmente... Isso pode levar um tempo.")

model_path = "/home/laza/chatbot_rag_hf/models/mistral-7b-instruct-v0.2.Q4_K_M.gguf"

llm = LlamaCpp(
    model_path=model_path,
    temperature=0.1,    # Controla a aleatoriedade da resposta
    max_tokens=512,     # Número máximo de tokens a serem gerados
    n_gpu_layers=20,    # VALOR CRÍTICO PARA SUA GPU - Ajuste conforme testes!
    n_batch=512,
    verbose=False,
    n_ctx=2048
)

print("Modelo GGUF carregado e pronto!")

def chatbot(query: str):
    base_prompt = custom_prompt(query)

    # --- MUDANÇA AQUI: Fortalecendo a instrução de idioma ---
    system_instruction = (
        "Você é um avaliador pedagógico da área de Matemática, claro, objetivo e embasado. "
        "Responda **APENAS EM PORTUGUÊS BRASILEIRO**, com base no contexto fornecido e de forma concisa."
        "Sua resposta deve ser sempre em português. Never answer in English."
    )
    # --- FIM DA MUDANÇA ---

    formatted_prompt = f"[INST] {system_instruction}\n\n{base_prompt} [/INST]"

    resposta = ""
    print("Bot: ", end="", flush=True)

    terminal_width = shutil.get_terminal_size().columns
    current_line_length = 0

    try:
        for chunk in llm.stream(formatted_prompt):
            content = chunk

            for char in content:
                print(char, end="", flush=True)
                current_line_length += 1

                if char == '\n':
                    current_line_length = 0
                elif current_line_length >= terminal_width:
                    print('\n', end="", flush=True)
                    current_line_length = 0
            
            resposta += content
        print()
    except Exception as e:
        print(f"\nErro ao gerar resposta: {e}")
        print("Não foi possível obter resposta.")
    
    return resposta

if __name__ == "__main__":
    print("Chatbot de Avaliação Pedagógica (Matemática)")
    print("Digite 'sair' para encerrar.")

    while True:
        user_query = input("Você: ")
        if user_query.lower() == 'sair':
            break
        
        response = chatbot(user_query)

Carregando modelo GGUF localmente... Isso pode levar um tempo.


llama_context: n_ctx_per_seq (2048) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


Modelo GGUF carregado e pronto!
